In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Acute_Myeloid_Leukemia"
cohort = "GSE121431"

# Input paths
in_trait_dir = "../../input/GEO/Acute_Myeloid_Leukemia"
in_cohort_dir = "../../input/GEO/Acute_Myeloid_Leukemia/GSE121431"

# Output paths
out_data_file = "../../output/preprocess/Acute_Myeloid_Leukemia/GSE121431.csv"
out_gene_data_file = "../../output/preprocess/Acute_Myeloid_Leukemia/gene_data/GSE121431.csv"
out_clinical_data_file = "../../output/preprocess/Acute_Myeloid_Leukemia/clinical_data/GSE121431.csv"
json_path = "../../output/preprocess/Acute_Myeloid_Leukemia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"SY-1365, a covalent, first in-class CDK7 inhibitor for cancer treatment"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['disease state: Acute Myeloid Leukemia'], 1: ['cell line: AML cell line  THP-1'], 2: ['agent: DMSO', 'agent: SY-1365', 'agent: JQ1', 'agent: NVP2', 'agent: FLAVO'], 3: ['time: 2 hours', 'time: 6 hours']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset likely contains gene expression data
# It's studying inhibitors for cancer treatment, which typically involves gene expression analysis
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Looking at the sample characteristics dictionary:
# - trait_row = 0 (disease state: Acute Myeloid Leukemia)
# - age_row is None (not available)
# - gender_row is None (not available)
trait_row = 0
age_row = None
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait value to binary (1 for AML, 0 for control)"""
    if value is None:
        return None
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # In this dataset, all samples appear to be AML
    if "Acute Myeloid Leukemia" in value:
        return 1
    return None

def convert_age(value):
    """Convert age value to continuous"""
    # Not used as age data is not available
    return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)"""
    # Not used as gender data is not available
    return None

# 3. Save Metadata
# Initial filtering on the usability of the dataset
# trait_row is not None, so trait data is available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path, 
                             is_gene_available=is_gene_available, 
                             is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we extract clinical features
if trait_row is not None:
    # Use clinical_data from previous step (assumed to be available)
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the clinical data
    preview = preview_df(selected_clinical_df)
    print("Clinical data preview:")
    print(preview)
    
    # Save the clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=True)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical data preview:
{'GSM3430924': [1.0], 'GSM3430925': [1.0], 'GSM3430926': [1.0], 'GSM3430927': [1.0], 'GSM3430928': [1.0], 'GSM3430929': [1.0], 'GSM3430930': [1.0], 'GSM3430931': [1.0], 'GSM3430932': [1.0], 'GSM3430933': [1.0], 'GSM3430934': [1.0], 'GSM3430935': [1.0], 'GSM3430936': [1.0], 'GSM3430937': [1.0], 'GSM3430938': [1.0], 'GSM3430939': [1.0], 'GSM3430940': [1.0], 'GSM3430941': [1.0], 'GSM3430942': [1.0], 'GSM3430943': [1.0], 'GSM3430944': [1.0], 'GSM3430945': [1.0], 'GSM3430946': [1.0], 'GSM3430947': [1.0], 'GSM3430948': [1.0], 'GSM3430949': [1.0], 'GSM3430950': [1.0], 'GSM3430951': [1.0], 'GSM3430952': [1.0], 'GSM3430953': [1.0]}
Clinical data saved to ../../output/preprocess/Acute_Myeloid_Leukemia/clinical_data/GSE121431.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at',
       '11715104_s_at', '11715105_at', '11715106_x_at', '11715107_s_at',
       '11715108_x_at', '11715109_at', '11715110_at', '11715111_s_at',
       '11715112_at', '11715113_x_at', '11715114_x_at', '11715115_s_at',
       '11715116_s_at', '11715117_x_at', '11715118_s_at', '11715119_s_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers appear to be Affymetrix probe IDs (format: #########_at, #########_s_at, #########_x_at)
# These are not human gene symbols and need to be mapped to gene symbols
requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at', '11715104_s_at'], 'GeneChip Array': ['Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array'], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': [40780.0, 40780.0, 40780.0, 40780.0, 40780.0], 'Sequence Type': ['Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database'], 'Transcript ID(Array Design)': ['g21264570', 'g21264570', 'g21264570', 'g22748780', 'g30039713'], 'Target Description': ['g21264570 /TID=g21264570 /CNT=1 /FEA=FLmRNA /TIER=FL /STK=0 /DEF=g21264570 /REP_O

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns for gene identifiers and gene symbols
# From the preview, we can see that 'ID' column contains probe IDs matching the gene expression data
# and 'Gene Symbol' column contains the gene symbols we need
# These correspond to the identifiers seen in gene_data.index

# 2. Get the gene mapping dataframe
prob_col = 'ID'
gene_col = 'Gene Symbol'
mapping_data = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 3. Apply the gene mapping to convert probe-level measurements to gene expression
gene_data = apply_gene_mapping(gene_data, mapping_data)

# Preview the result
print("Mapped gene data preview - first 20 gene symbols:")
print(gene_data.index[:20])
print(f"Shape of gene expression data: {gene_data.shape}")


Mapped gene data preview - first 20 gene symbols:
Index(['A1BG', 'A1CF', 'A2LD1', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAA1',
       'AAAS', 'AACS', 'AACSP1', 'AADAC', 'AADACL2', 'AADACL3', 'AADACL4',
       'AADAT', 'AAGAB', 'AAK1', 'AAMP', 'AANAT'],
      dtype='object', name='Gene')
Shape of gene expression data: (19534, 30)


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the previously saved clinical data instead of re-extracting it
clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
print(f"Clinical data loaded from {out_clinical_data_file}")
print(f"Clinical data shape: {clinical_df.shape}")
print("Clinical data preview:")
print(preview_df(clinical_df))

# 3. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# Verify that the trait column has at least two unique values
unique_trait_values = linked_data[trait].unique()
print(f"Unique values in trait column: {unique_trait_values}")

# 5. Determine whether the trait and some demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct quality check and save the cohort information
note = "Dataset contains only AML (Acute Myeloid Leukemia) samples, which may limit certain analyses."
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=note
)

# 7. If the linked data is usable, save it
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Processed dataset saved to {out_data_file}")
else:
    print("Dataset not usable due to bias in trait distribution. Data not saved.")

Normalized gene data saved to ../../output/preprocess/Acute_Myeloid_Leukemia/gene_data/GSE121431.csv
Clinical data loaded from ../../output/preprocess/Acute_Myeloid_Leukemia/clinical_data/GSE121431.csv
Clinical data shape: (1, 30)
Clinical data preview:
{'GSM3430924': [1.0], 'GSM3430925': [1.0], 'GSM3430926': [1.0], 'GSM3430927': [1.0], 'GSM3430928': [1.0], 'GSM3430929': [1.0], 'GSM3430930': [1.0], 'GSM3430931': [1.0], 'GSM3430932': [1.0], 'GSM3430933': [1.0], 'GSM3430934': [1.0], 'GSM3430935': [1.0], 'GSM3430936': [1.0], 'GSM3430937': [1.0], 'GSM3430938': [1.0], 'GSM3430939': [1.0], 'GSM3430940': [1.0], 'GSM3430941': [1.0], 'GSM3430942': [1.0], 'GSM3430943': [1.0], 'GSM3430944': [1.0], 'GSM3430945': [1.0], 'GSM3430946': [1.0], 'GSM3430947': [1.0], 'GSM3430948': [1.0], 'GSM3430949': [1.0], 'GSM3430950': [1.0], 'GSM3430951': [1.0], 'GSM3430952': [1.0], 'GSM3430953': [1.0]}
Linked data shape: (30, 19327)


Data shape after handling missing values: (30, 19327)
Unique values in trait column: [1.]
Quartiles for 'Acute_Myeloid_Leukemia':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Acute_Myeloid_Leukemia' in this dataset is severely biased.

A new JSON file was created at: ../../output/preprocess/Acute_Myeloid_Leukemia/cohort_info.json
Dataset not usable due to bias in trait distribution. Data not saved.
